In [1]:
import os
from PIL import Image
import numpy as np
from numpy.linalg import norm, pinv
np.set_printoptions(precision=12, suppress=True)

In [2]:
consider_distort = False # flag whether to consider distortion
zoomed_coords = True # whether to divide coordinates by 5 or not (zoomed coords or not)

In [3]:
folder_name = 'data/star_coords/2016nov_txt/' # folder with coords files

In [4]:
# 20160218
# fnames = [
#     '20160218-201934-437.txt',
#     '20160218-211934-484.txt'
# ]

# 20160219
# fnames = [
#     '20160219-001934-484.txt',
#     '20160219-021934-593.txt',
#     '20160219-041934-593.txt'
# ]

# 20160311
# fnames = [
#     '20160311-020811-125.txt',
#     '20160311-030811-125.txt'
# ]

# 20161122 Nov-11
# fnames = [
#     '20161122-221517-375.txt'
# ]

# 20161121
fnames = [
    '20161121-220921-250.txt'
]

In [5]:
if consider_distort:
    print "consider_distort is True!"

In [6]:
coords_list = []
for fname in fnames:
    piece = np.loadtxt(folder_name + os.sep + fname)
    coords_list.append(piece)

coords = np.vstack(coords_list)
print 'Zoomed In Star coordinates pairs:\n', coords, '\n'

if zoomed_coords:
    coords /= 5.0
    coords = coords.round()
    print 'Normal Star coordinates pairs:\n', coords

Zoomed In Star coordinates pairs:
[[  907.  1674.  2124.  1893.]
 [  795.  2723.  2033.  2932.]
 [ 1511.  3124.  2734.  3321.]
 [ 5242.  1219.  6425.  1369.]
 [ 5587.  4718.  6823.  4900.]
 [ 4552.  4498.  5766.  4678.]] 

Normal Star coordinates pairs:
[[  181.   335.   425.   379.]
 [  159.   545.   407.   586.]
 [  302.   625.   547.   664.]
 [ 1048.   244.  1285.   274.]
 [ 1117.   944.  1365.   980.]
 [  910.   900.  1153.   936.]]


In [7]:
lX = coords[:, 0] # leftX coordinates
lY = coords[:, 1] # leftY coordinates
rX = coords[:, 2] # rightX coordinates
rY = coords[:, 3] # rightY coordinates

N = coords.shape[0] # number of pairs of points
M = coords.shape[1] # lX, lY, rX, rY == 4
print 'Number of Star coordinates pairs:', N

Number of Star coordinates pairs: 6


In [8]:
xi = np.zeros(2*N)

# First, calculate E,F
num_coeffs = 2
if consider_distort:
    num_coeffs = 8

z = np.zeros(num_coeffs)
arr = np.zeros((2*N, num_coeffs)) # matrix A
print 'Number of coefficients:', num_coeffs

Number of coefficients: 2


In [9]:
for i in xrange(N): # fill the xi vector
    xi[2*i] = rX[i] - lX[i]
    xi[2*i + 1] = rY[i] - lY[i]

In [10]:
if consider_distort:
    im = Image.open(folder_name + 'mod_'+ fnames[0][:-4] +'-1.png')
    x_c = im.width / 2.0
    y_c = im.height / 2.0
    for i in xrange(N): # fill the A matrix
        dist_l = (lX[i]-x_c)**2 + (lY[i]-y_c)**2
        dist_r = (rX[i]-x_c)**2 + (rY[i]-y_c)**2

        zx1 = (lX[i] - x_c) * dist_l
        zx2 = (rX[i] - x_c) * dist_r
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0, -zx1, zx2]

        zy1 = (lY[i] - y_c) * dist_l
        zy2 = (rY[i] - y_c) * dist_r
        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1, -zy1, zy2]

else:
    for i in xrange(N): # fill the A matrix
        arr[2*i] = [1, 0]

        arr[2*i + 1] = [0, 1]

In [11]:
np.set_printoptions(precision=2, suppress=True)
print 'A:\n', arr, '\n'
print 'xi:\n', xi
np.set_printoptions(precision=12, suppress=True)

A:
[[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]] 

xi:
[ 244.   44.  248.   41.  245.   39.  237.   30.  248.   36.  243.   36.]


In [12]:
p_arr = pinv(arr)
z = np.dot(p_arr, xi)
ef = z
e = z[0]
f = z[1]
print 'Affine coefficients EF: \n', z

Affine coefficients EF: 
[ 244.166666666667   37.666666666667]


In [13]:
# Second, calculate ABCD
num_coeffs = 4

z = np.zeros(num_coeffs)
arr = np.zeros((2*N, num_coeffs)) # matrix A
print 'Number of coefficients:', num_coeffs

Number of coefficients: 4


In [14]:
for i in xrange(N): # fill the xi vector
    xi[2*i] = rX[i] - e
    xi[2*i + 1] = rY[i] - f

In [15]:
if consider_distort:
    im = Image.open(folder_name + 'mod_'+ fnames[0][:-4] +'-1.png')
    x_c = im.width / 2.0
    y_c = im.height / 2.0
    for i in xrange(N): # fill the A matrix
        dist_l = (lX[i]-x_c)**2 + (lY[i]-y_c)**2
        dist_r = (rX[i]-x_c)**2 + (rY[i]-y_c)**2

        zx1 = (lX[i] - x_c) * dist_l
        zx2 = (rX[i] - x_c) * dist_r
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0, -zx1, zx2]

        zy1 = (lY[i] - y_c) * dist_l
        zy2 = (rY[i] - y_c) * dist_r
        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1, -zy1, zy2]

else:
    for i in xrange(N): # fill the A matrix
        arr[2*i] = [lX[i], lY[i], 0, 0]

        arr[2*i + 1] = [0, 0, lX[i], lY[i]]

In [16]:
np.set_printoptions(precision=2, suppress=True)
print 'A:\n', arr, '\n'
print 'xi:\n', xi
np.set_printoptions(precision=12, suppress=True)

A:
[[  181.   335.     0.     0.]
 [    0.     0.   181.   335.]
 [  159.   545.     0.     0.]
 [    0.     0.   159.   545.]
 [  302.   625.     0.     0.]
 [    0.     0.   302.   625.]
 [ 1048.   244.     0.     0.]
 [    0.     0.  1048.   244.]
 [ 1117.   944.     0.     0.]
 [    0.     0.  1117.   944.]
 [  910.   900.     0.     0.]
 [    0.     0.   910.   900.]] 

xi:
[  180.83   341.33   162.83   548.33   302.83   626.33  1040.83   236.33
  1120.83   942.33   908.83   898.33]


In [17]:
p_arr = pinv(arr)
z = np.dot(p_arr, xi)
abcd = z
print 'Affine coefficients ABCD: \n', z

Affine coefficients ABCD: 
[ 0.992938444278  0.008116357104 -0.009651269154  1.009224581851]


In [18]:
z = np.hstack([abcd, ef])
print 'Affine coefficients: \n', z

Affine coefficients: 
[   0.992938444278    0.008116357104   -0.009651269154    1.009224581851
  244.166666666667   37.666666666667]


In [19]:
sigma = 2 # 2 pix error
I = np.eye(2*N)

print 'cond(A): ', np.linalg.cond(arr)
proba = (2*N - num_coeffs) / norm( np.dot(I - np.dot(arr, p_arr), xi) )**2
sig_theory = (1 / proba)**0.5

print 'Probability:', proba
print 'Sigma theory:', sig_theory
print 'Sigma practice', sigma

cond(A):  3.48580982428
Probability: 0.133654036855
Sigma theory: 2.73532515671
Sigma practice 2


In [20]:
res_fname = 'results/2016nov/'

if consider_distort:
    res_fname += 'affdis_coeffs_shots'
else:
    res_fname += 'affine_coeffs_shots'

res_fname += str(len(fnames)) + '_' + fnames[0]

In [21]:
np.savetxt(res_fname, z, fmt="%.10f")
print 'Coeffs are saved to:', res_fname

Coeffs are saved to: results/2016nov/affine_coeffs_shots1_20161121-220921-250.txt
